# Семинар 8. Практическая работа.
На сайте запущен А/В тест с целью увеличить доход.  
В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id,  
тип выборки variant_name и доход принесенный пользователем revenue.  
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [82]:
import pandas as pd
import numpy as np
from scipy import stats

In [83]:
df = pd.read_csv(r'AB_Test_Results.csv', sep=';')

In [84]:
df.head(2)

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,"0,00"
1,2423,control,"0,00"


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   USER_ID       10000 non-null  int64 
 1   VARIANT_NAME  10000 non-null  object
 2    REVENUE      10000 non-null  object
dtypes: int64(1), object(2)
memory usage: 234.5+ KB


In [86]:
df.rename(columns={' REVENUE ': 'REVENUE'}, inplace=True)

In [87]:
df['REVENUE'] = df['REVENUE'].str.replace(',', '.').astype('float')

In [88]:
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [89]:
df_new = df
df_new["purchase"]=np.where(df_new['REVENUE']>0, 1, 0)
df_new


,USER_ID,VARIANT_NAME,REVENUE,purchase
0,737,variant,0.0,0
1,2423,control,0.0,0
2,9411,control,0.0,0
3,7311,control,0.0,0
4,6174,variant,0.0,0
...,...,...,...,...
9995,1981,control,0.0,0
9996,502,variant,0.0,0
9997,9214,variant,0.0,0
9998,7741,control,0.0,0


In [90]:
df_new['USER_ID'].value_counts()

5652    6
8359    6
668     6
9101    6
4879    6
       ..
5455    1
1399    1
6967    1
3156    1
9468    1
Name: USER_ID, Length: 6324, dtype: int64

In [91]:
s1 = pd.merge(df_new[df_new["VARIANT_NAME"]=="variant"], df_new[df_new["VARIANT_NAME"]=="control"], how='inner', on=['USER_ID'])
s1["USER_ID"]

0       6205
1       6205
2       2529
3       2529
4       2529
        ... 
2496    4034
2497    8471
2498    8471
2499    4883
2500    9303
Name: USER_ID, Length: 2501, dtype: int64

In [92]:
df_new_2 = df_new[df_new.USER_ID.isin(s1["USER_ID"]) == False ]
df_new_2

,USER_ID,VARIANT_NAME,REVENUE,purchase
0,737,variant,0.0,0
4,6174,variant,0.0,0
5,2380,variant,0.0,0
7,9168,control,0.0,0
9,7548,control,0.0,0
...,...,...,...,...
9993,2400,variant,0.0,0
9994,3129,control,0.0,0
9996,502,variant,0.0,0
9998,7741,control,0.0,0


In [93]:
df_new_2.VARIANT_NAME.value_counts()

variant    3044
control    3026
Name: VARIANT_NAME, dtype: int64

In [94]:
stats.mannwhitneyu(x=df_new_2[(df_new_2['VARIANT_NAME'] == 'control')]['REVENUE'].values,
                                                  y=df_new_2[(df_new_2['VARIANT_NAME'] == 'variant')]['REVENUE'].values)[1]


0.2444173738649208

In [95]:
stats.mannwhitneyu(x=df_new_2[(df_new_2['VARIANT_NAME'] == 'control')]['purchase'].values,
                                                  y=df_new_2[(df_new_2['VARIANT_NAME'] == 'variant')]['purchase'].values)[1]

0.24799567813991064

## Вывод:
На основание результатов тестов, проведенных на очищенных данных,  
можно сделать вывод о том что различий между группами нет.  
Следовательно, изменения не повлияли на доход.  

Но считаю не корректным делать выводы на основе этих данных, так как  
наблюдается некорректная работу алгоритма сплитования,  
примерно сорок процентов данных это дубли и пользователи которые попали в обе группы одновременно.  
Рекомендую доработать алгоритм сплитования, проверить его работу с помощью А/А теста.  
И после этого запустить повторный А/В тест.  